In [ ]:
import numpy as np
import math as m
path = 'input.txt'

matrix = np.array([np.array(list(line)).astype(np.int8) for line in open(path).read().splitlines()])

**Astar**
- https://medium.com/@nicholas.w.swift/easy-a-star-pathfinding-7e6689c7f7b2

In [ ]:
def heuristic_chebyshev(node, goal):
    # Chebyshev Distance heuristic
    return max(abs(node[0] - goal[0]), abs(node[1] - goal[1]))

def heuristic_octile(node, goal):
    # Octile Distance heuristic
    return max(abs(node[0] - goal[0]), abs(node[1] - goal[1])) + (2**0.5 - 1) * min(abs(node[0] - goal[0]), abs(node[1] - goal[1]))

def heuristic_manhattan_tie_breaking(node, goal):
    # Manhattan Distance with Ties Breaking heuristic
    return abs(node[0] - goal[0]) + abs(node[1] - goal[1]) + 0.1 * min(abs(node[0] - goal[0]), abs(node[1] - goal[1]))

def heuristic_euclidean_squared(node, goal):
    # Euclidean Distance Squared heuristic
    return ((node[0] - goal[0])**2 + (node[1] - goal[1])**2)

def heuristic_diagonal_distance(node, goal):
    # Diagonal Distance heuristic
    return max(abs(node[0] - goal[0]), abs(node[1] - goal[1])) + (2**0.5 - 1) * min(abs(node[0] - goal[0]), abs(node[1] - goal[1]))

def heuristic_euclidean(node, goal):
    # Euclidean distance heuristic
    return ((node[0] - goal[0])**2 + (node[1] - goal[1])**2)

def heuristic_manhattan(node, goal):
    # A simple postive Manhattan distance heuristic 
    return (abs(node[0] - goal[0]) + abs(node[1] - goal[1])) * 0.5

def heuristic(node,goal):
    return heuristic_octile(node, goal)

In [ ]:
import tkinter as tk
from tkinter import ttk

import heapq

class Node:

    def __init__(self, x, y, cost=0, heuristic=0, dir=None, steps=1):
        self.x = x
        self.y = y
        self.cost = cost
        self.heuristic = heuristic
        self.total_cost = cost + heuristic
        self.parent = None
        self.dir = dir
        self.steps = steps

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def __hash__(self):
        return hash((self.x, self.y))
    
    def __lt__(self, other):
        return self.total_cost < other.total_cost


def direction(x, y):
    if x == 0 and y == -1:
        return "North"
    elif x == 0 and y == 1:
        return "South"
    elif x == 1 and y == 0:
        return "East"
    elif x == -1 and y == 0:
        return "West"
    else:
        return "Unknown"

def get_neighbors(node):
    x, y = node.x, node.y
    neighbors = []

    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            if dx == 0 and dy == 0:
                continue
            if dx != 0 and dy != 0:
                continue  # Skip diagonal neighbors
            
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < len(matrix) and 0 <= new_y < len(matrix[0]):
                neighbors.append((new_x, new_y))
    return neighbors

def astar(start, end):
    open_set = []
    closed_set = {}
    low = 1000000000

    h_score = heuristic(start, end)
    start_node = Node(*start, heuristic=h_score,dir='South', cost=matrix[0][0])
    start_node2 = Node(*start, heuristic=h_score,dir='East', cost=matrix[0][0])
    end_node = Node(*end)

    heapq.heappush(open_set, (start_node.total_cost, start_node))
    heapq.heappush(open_set, (start_node2.total_cost, start_node2))

    while open_set:
        #makes sure to get the smalles cost one
        current_node = heapq.heappop(open_set)[1]
        if current_node == end_node:
            path = []
            sum = current_node.cost
            while current_node:
                path.append((current_node.x, current_node.y))
                array_copy[current_node.y][current_node.x] = 'X'
                current_node = current_node.parent

            close_window()
            return sum 
            
        coordinates = (current_node.x, current_node.y)
        if coordinates in closed_set:
            existing_node = closed_set[coordinates]

            # Compare costs and update if necessary
            if current_node < existing_node:
                closed_set[coordinates] = current_node
        else:
            closed_set[coordinates] = current_node
        
        
        if current_node.heuristic < low:
            low = current_node.heuristic
            progress = 100 - (current_node.heuristic / start_node.heuristic) * 100
            progress_bar["value"] = progress
            progress_bar.update()

        for neighbor in get_neighbors(current_node):
            g_score = current_node.cost + matrix[neighbor[1]][neighbor[0]]
            h_score = heuristic(neighbor, (end_node.x,end_node.y))
            f_score = g_score + h_score
            dir = direction(neighbor[0] - current_node.x, neighbor[1] - current_node.y)

            if current_node.dir == dir and current_node.steps == 3:
                continue
            elif current_node.dir != dir:
                steps = 1
            else:
                steps = current_node.steps + 1

            if (current_node.dir == 'West' and dir == 'East') or (current_node.dir == 'East' and dir == 'West') or \
                (current_node.dir == 'North' and dir == 'South') or (current_node.dir == 'South' and dir == 'North'):
                continue

            new_node = Node(*neighbor, cost=g_score, heuristic=h_score, dir=dir, steps=steps)
            new_node.parent = current_node

            if any(neighbor_node[1].x == new_node.x and neighbor_node[1].y == new_node.y\
                   and neighbor_node[1].cost < new_node.cost for neighbor_node in open_set):
                continue

            heapq.heappush(open_set, (f_score, new_node))

array_copy = np.array([np.array(['.' for char in line]).astype(object) for line in matrix])
start = (0, 0)
end = (len(matrix[0])-1, len(matrix)-1)

# Create a Tkinter window
root = tk.Tk()
root.title("A* Progress Bar Example")
def close_window():
    root.destroy()

# Create a progress bar
progress_bar = ttk.Progressbar(root, length=200, mode="determinate")
progress_bar.pack(pady=10)




print(astar(start, end))

for line in array_copy:
    print(line)

In [ ]:
import heapq

class Node:

    def __init__(self, x, y, cost=0, dir=None, steps=1):
        self.x = x
        self.y = y
        self.cost = cost
        self.parent = None
        self.dir = dir
        self.steps = steps

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def __hash__(self):
        return hash((self.x, self.y))
    
    def __lt__(self, other):
        return self.cost < other.cost

def direction(x, y):
    if x == 0 and y == -1:
        return "North"
    elif x == 0 and y == 1:
        return "South"
    elif x == 1 and y == 0:
        return "East"
    elif x == -1 and y == 0:
        return "West"
    else:
        return "Unknown"

def get_neighbors(node):
    x, y = node.x, node.y
    neighbors = []

    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            if dx == 0 and dy == 0:
                continue
            if dx != 0 and dy != 0:
                continue  # Skip diagonal neighbors
            
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < len(matrix[0]) and 0 <= new_y < len(matrix):
                neighbors.append((new_x, new_y))
    return neighbors

def dijkstra(start, end):
    open_set = []
    closed_set = set()

    start_node = Node(*start, cost=matrix[0][0])
    end_node = Node(*end)

    heapq.heappush(open_set, start_node)

    while open_set:
        current_node = heapq.heappop(open_set)
        if current_node == end_node:
            path = []
            sum = current_node.cost
            while current_node:
                path.append((current_node.x, current_node.y))
                array_copy[current_node.y][current_node.x] = 'O'
                current_node = current_node.parent
            return sum
            
        if current_node in closed_set:
            continue

        closed_set.add(current_node) 
        
        for neighbor in get_neighbors(current_node):
            g_score = current_node.cost + matrix[neighbor[1]][neighbor[0]]
            dir = direction(neighbor[0] - current_node.x, neighbor[1] - current_node.y)

            if current_node.dir == dir and current_node.steps == 3:
                continue
            elif current_node.dir != dir:
                steps = 1
            else:
                steps = current_node.steps + 1

            if (current_node.dir == 'West' and dir == 'East') or (current_node.dir == 'East' and dir == 'West') or \
                (current_node.dir == 'North' and dir == 'South') or (current_node.dir == 'South' and dir == 'North'):
                continue

            new_node = Node(*neighbor, cost=g_score, dir=dir, steps=steps)
            new_node.parent = current_node

            heapq.heappush(open_set, new_node)

# Example usage:
start = (0, 0)
end = (len(matrix[0]) - 1, len(matrix) - 1)
array_copy = np.array([np.array(['.' for char in line]).astype(object) for line in matrix])

result = dijkstra(start, end)
print(result)

for line in array_copy:
    print(line)

In [ ]:
import heapq

#second try at a-star
class Node:
    def __init__(self, y, x, cost=0, dir=None, prarent=None):
        self.y = y
        self.x = x
        self.cost = cost
        self.parent = prarent
        self.dir = dir

    def __lt__(self, other):
        return self.cost < other.cost
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y and self.dir == other.dir
    
    def __hash__(self):
        return hash((self.x, self.y, self.dir))

def dijkstra(matrix, min=1, max=3):
    seen = []
    discovered = set()
    max_y, max_x = (v - 1 for v in matrix.shape)
    
    start = 0, 0
    goal = max_y, max_x

    start_node = Node(*start, dir=0)
    start_node2 = Node(*start, dir=1)

    end_node = Node(*goal)

    heapq.heappush(seen, start_node)
    heapq.heappush(seen, start_node2)


    while seen:
        current_node = heapq.heappop(seen)
        if current_node.x == end_node.x and current_node.y == end_node.y:
            cost = current_node.cost
            while current_node:
                array_copy[current_node.y][current_node.x] = 'O'
                current_node = current_node.parent
            return cost
        

        if current_node in discovered:
            continue

        discovered.add(current_node)

        dir = current_node.dir
        original_cost = current_node.cost
        for s in [-1, 1]:    
            cost = original_cost
            new_y, new_x = current_node.y, current_node.x
            new_node = current_node
            for i in range(1, max + 1):
                if current_node.dir == 1:
                    new_x = current_node.x + i * s
                else:
                    new_y = current_node.y + i * s

                if new_x < 0 or new_y < 0 or new_x > max_x or new_y > max_y:
                    break

                cost += matrix[new_y, new_x]
                new_node = Node(new_y, new_x, prarent=new_node, cost=cost, dir=1-dir)
                
                if new_node in discovered:
                    continue

                if i >= min:
                    heapq.heappush(seen, new_node)


array_copy = np.array([np.array(['.' for char in line]).astype(object) for line in matrix])

print(dijkstra(matrix))

In [ ]:
import heapq
def navigate(grid, minval=1, maxval=3):
    q = []
    max_y, max_x = (v - 1 for v in grid.shape)
    goal = max_y, max_x
    heapq.heappush(q,(0, (0, 0, 0)))
    heapq.heappush(q, (0, (0, 0, 1)))
    seen = set()
    
    while q:
        cost, (y, x, direction) = heapq.heappop(q)

        if (y, x) == goal:
            break

        if (y, x, direction) in seen:
            continue

        seen.add((y, x, direction))
        original_cost = cost
        for s in [-1,1]:
            cost = original_cost
            new_y, new_x = y, x
            for i in range(1, maxval + 1):
                if direction == 1:
                    new_x = x + i * s
                else:
                    new_y = y + i * s

                if new_x < 0 or new_y < 0 or new_x > max_x or new_y > max_y:
                    break
                cost += grid[new_y, new_x]
                if ((new_y, new_x, 1 - direction)) in seen:
                    continue
                if i >= minval:
                    heapq.heappush(q,(cost, (new_y, new_x, 1 - direction)))
    return cost

navigate(matrix, 4,10)
